# Retracting adiabatically a ferromagnetic domain 

Here we are going to reduce the ferromagnetic domain just like we incresed the domian but by tuning the magnetic field in the inverse manner. 

In [23]:
import numpy as np
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from ising_kitaev import initialize_chain, run_adiabatic_zeeman_change, rotate_to_measurement_basis, add_measurement

## Setup

### Parameters

Here we use the same bounds of magnetic field , but we are increasing magnetic field on the first site

In [24]:
zeeman_ferro = 0.01
zeeman_para = 10

In [25]:
initial_config = np.array([zeeman_ferro, zeeman_ferro, zeeman_ferro, zeeman_ferro])
final_config = np.array([zeeman_para, zeeman_ferro, zeeman_ferro, zeeman_ferro])

### Initialize the circuit

In [26]:
qreg = QuantumRegister(4)
creg = ClassicalRegister(3)
qcirc = QuantumCircuit(qreg, creg)

In [27]:
initialize_chain(qcirc, qreg, initial_config, 'logical_one')
qcirc.draw()

### Move the chain by one site

We evolve the Zeeman slowly and in between each update of the Zeeman we run the dynamic of the chain using trotter for t such that J*t = 1

In [28]:
run_adiabatic_zeeman_change(qcirc, qreg, initial_config, final_config, 0, 0.25, 0.25, 2, 20)
qcirc.depth()

5743

### Add the measurement

We perform the measurement in the logical basis.

In [29]:
rotate_to_measurement_basis(qcirc, qreg, [1, 2, 3])
add_measurement(qcirc, qreg, creg, [1, 2, 3])

## Run the simulation

In [30]:
from qiskit import Aer, execute

In [31]:
backend = Aer.get_backend('qasm_simulator')
job = execute(qcirc, backend, shots=2000)

In [32]:
job.status()

<JobStatus.RUNNING: 'job is actively running'>

In [33]:
result = job.result()
print(result.get_counts())

{'000': 5, '001': 1995}


As expected we get a single peak corresponding to the state where we have a paramagnetic phase in the first site